# Deep-Live-Cam on Google Colab
Real-time face swap with Gradio interface

In [ ]:
# Install system dependencies
!apt update -qq
!apt install -y ffmpeg

# Clone repository
!git clone https://github.com/Mayank-kanojiya/liveswap.git
%cd liveswap

In [ ]:
# Install Python dependencies
!pip install -q gradio opencv-python insightface onnxruntime-gpu torch torchvision
!pip install -q customtkinter pillow psutil opennsfw2 protobuf cv2-enumerate-cameras
!pip install -q git+https://github.com/xinntao/BasicSR.git@master
!pip install -q git+https://github.com/TencentARC/GFPGAN.git@master

In [ ]:
# Download required models
import os
os.makedirs('models', exist_ok=True)

!wget -q -O models/GFPGANv1.4.pth https://huggingface.co/hacksider/deep-live-cam/resolve/main/GFPGANv1.4.pth
!wget -q -O models/inswapper_128.onnx https://huggingface.co/hacksider/deep-live-cam/resolve/main/inswapper_128_fp16.onnx

In [ ]:
# Create Gradio interface
import gradio as gr
import cv2
import tempfile
import sys
import os
os.environ['DISPLAY'] = ':0'
sys.path.append('.')

# Initialize modules globals first
import modules.globals
modules.globals.headless = True
modules.globals.execution_providers = ['CPUExecutionProvider']
modules.globals.frame_processors = ['face_swapper']

# Import face swap modules
from modules.processors.frame.face_swapper import get_face_swapper, process_frame
from modules.face_analyser import get_one_face

# Initialize face swapper
face_swapper = get_face_swapper()

def swap_face(source_image, target_image):
    if source_image is None or target_image is None:
        return None, "Please upload both source and target images"
    
    try:
        # Read images
        source_img = cv2.imread(source_image)
        target_img = cv2.imread(target_image)
        
        if source_img is None:
            return None, "Could not read source image"
        if target_img is None:
            return None, "Could not read target image"
        
        # Get source face
        source_face = get_one_face(source_img)
        if source_face is None:
            return None, "No face detected in source image"
        
        # Process frame with face swapper
        result_frame = process_frame(source_face, target_img, face_swapper)
        if result_frame is None or (result_frame == target_img).all():
            return None, "Face swap failed - no face detected in target image"
        
        # Save result
        result_path = tempfile.mktemp(suffix='.jpg')
        cv2.imwrite(result_path, result_frame)
        
        return result_path, "Face swap completed successfully!"
        
    except Exception as e:
        return None, f"Error: {str(e)}"

# Create interface
with gr.Blocks(title="Deep-Live-Cam") as demo:
    gr.Markdown("# Deep-Live-Cam - Face Swap")
    
    with gr.Row():
        with gr.Column():
            source_image = gr.Image(label="Source Face Image", type="filepath")
            target_image = gr.Image(label="Target Image", type="filepath")
            swap_btn = gr.Button("Swap Face", variant="primary")
            
        with gr.Column():
            result_media = gr.File(label="Result")
            status_text = gr.Textbox(label="Status", interactive=False)
    
    swap_btn.click(
        fn=swap_face,
        inputs=[source_image, target_image],
        outputs=[result_media, status_text]
    )

demo.launch(share=True, debug=True)